In [29]:
%matplotlib inline

import accelerate
#import numbapro
import mkl
#import iopro

# General libraries.
from __future__ import division
from collections import Counter
import csv
import dateutil
import os
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt

# SK-learn libraries for preprocessing.
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder # for integer values
from sklearn.feature_extraction import DictVectorizer as DV
from sklearn.cross_validation import StratifiedShuffleSplit

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.mixture import GMM
from sklearn.grid_search import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import TransformerMixin


# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

In [2]:
data_path = "data"
submissions_path = "submissions"
if not data_path or not submissions_path:
    raise Exception("Set the data and submission paths in competition_utilities.py!")

def parse_date_maybe_null(date):
    if date:
        return dateutil.parser.parse(date)
    return None

df_converters = {"Dates": dateutil.parser.parse}

def get_reader(file_name="train.csv"):
    reader = csv.reader(open(os.path.join(data_path, file_name)))
    header = reader.next()
    return reader

def get_header(file_name="train.csv"):
    reader = csv.reader(open(os.path.join(data_path, file_name)))
    header = reader.next()
    return header

def get_dataframe(file_name="train.csv"):
    return pd.io.parsers.read_csv(os.path.join(data_path, file_name), converters = df_converters)

    
def write_submission(file_name, predictions):
    writer = csv.writer(open(os.path.join(submissions_path, file_name), "w"), lineterminator="\n")
    writer.writerows(predictions)    

In [3]:
TestD = get_dataframe("test.csv")
TestD.head()

,Id,Dates,DayOfWeek,PdDistrict,Address,X,Y
0,0,2015-05-10 23:59:00,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051
1,1,2015-05-10 23:51:00,Sunday,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432
2,2,2015-05-10 23:50:00,Sunday,NORTHERN,2000 Block of GOUGH ST,-122.426002,37.792212
3,3,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412
4,4,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412


In [6]:
test_data = TestD.copy(deep=True)

In [7]:
TrainD = get_dataframe("train.csv")
TrainD.head()

,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541


In [14]:
train_data = TrainD.copy(deep=True)

In [15]:
number_of_categories = train_data['Category'].nunique()
category_names = sorted(train_data['Category'].unique())
print number_of_categories
print category_names

39
['ARSON', 'ASSAULT', 'BAD CHECKS', 'BRIBERY', 'BURGLARY', 'DISORDERLY CONDUCT', 'DRIVING UNDER THE INFLUENCE', 'DRUG/NARCOTIC', 'DRUNKENNESS', 'EMBEZZLEMENT', 'EXTORTION', 'FAMILY OFFENSES', 'FORGERY/COUNTERFEITING', 'FRAUD', 'GAMBLING', 'KIDNAPPING', 'LARCENY/THEFT', 'LIQUOR LAWS', 'LOITERING', 'MISSING PERSON', 'NON-CRIMINAL', 'OTHER OFFENSES', 'PORNOGRAPHY/OBSCENE MAT', 'PROSTITUTION', 'RECOVERED VEHICLE', 'ROBBERY', 'RUNAWAY', 'SECONDARY CODES', 'SEX OFFENSES FORCIBLE', 'SEX OFFENSES NON FORCIBLE', 'STOLEN PROPERTY', 'SUICIDE', 'SUSPICIOUS OCC', 'TREA', 'TRESPASS', 'VANDALISM', 'VEHICLE THEFT', 'WARRANTS', 'WEAPON LAWS']


In [16]:
labels = train_data["Category"].astype('category')
if "Category" in train_data.columns:
    del train_data['Category']

0                       WARRANTS
1                 OTHER OFFENSES
2                 OTHER OFFENSES
3                  LARCENY/THEFT
4                  LARCENY/THEFT
5                  LARCENY/THEFT
6                  VEHICLE THEFT
7                  VEHICLE THEFT
8                  LARCENY/THEFT
9                  LARCENY/THEFT
10                 LARCENY/THEFT
11                OTHER OFFENSES
12                     VANDALISM
13                 LARCENY/THEFT
14                  NON-CRIMINAL
15                  NON-CRIMINAL
16                       ROBBERY
17                       ASSAULT
18                OTHER OFFENSES
19                  NON-CRIMINAL
20                 LARCENY/THEFT
21                       ROBBERY
22                      WARRANTS
23                  NON-CRIMINAL
24                 LARCENY/THEFT
25                  NON-CRIMINAL
26                 LARCENY/THEFT
27                 LARCENY/THEFT
28                 LARCENY/THEFT
29                OTHER OFFENSES
          

# Features

### MAKE REGIONS FROM lat/long 

In [17]:
# Train data:

min_x, max_x = np.min(train_data['X']), np.max(train_data['X'])
min_y, max_y = np.min(train_data['Y']), np.max(train_data['Y'])
print "Original max and min values"
print "---------------------------"
print (min_x, max_x,min_y, max_y)


# fix invalid values:
train_data.loc[train_data['X'] > -122.3, 'X'] = -122.3
train_data.loc[train_data['Y'] > 37.8, 'Y'] = 37.8

grid_size = 10
grid_width = np.max(train_data['X']) - np.min(train_data['X'])
grid_height = np.max(train_data['Y']) - np.min(train_data['Y'])

x_interval = grid_width/grid_size
y_interval = grid_height/grid_size

min_x, max_x = np.min(train_data['X']), np.max(train_data['X'])
min_y, max_y = np.min(train_data['Y']), np.max(train_data['Y'])

train_data['region'] = (10*(np.ceil((train_data['X'] - min_x)/x_interval)) + (np.ceil((train_data['Y'] - min_y)/y_interval))).astype(int)

print "New max and min values"
print "---------------------------"
print (min_x, max_x, min_y, max_y)

Original max and min values
---------------------------
(-122.51364206429, -120.5, 37.707879022413501, 90.0)
New max and min values
---------------------------
(-122.51364206429, -122.3, 37.707879022413501, 37.799999999999997)


In [18]:
# Test data:

min_x, max_x = np.min(test_data.loc[:,'X']), np.max(test_data.loc[:,'X'])
min_y, max_y = np.min(test_data.loc[:,'Y']), np.max(test_data.loc[:,'Y'])

print "Original max and min values"
print "---------------------------"
print (min_x, max_x,min_y, max_y)

test_data.loc[test_data['X'] > -122.3, 'X'] = -122.3
test_data.loc[test_data['Y'] > 37.8, 'Y'] = 37.8

grid_size = 10
grid_width = np.max(test_data.loc[:,'X']) - np.min(test_data.loc[:,'X'])
grid_height = np.max(test_data.loc[:,'Y']) - np.min(test_data.loc[:,'Y'])

x_interval = grid_width/grid_size
y_interval = grid_height/grid_size

min_x, max_x = np.min(test_data.loc[:,'X']), np.max(test_data.loc[:,'X'])
min_y, max_y = np.min(test_data.loc[:,'Y']), np.max(test_data.loc[:,'Y'])

test_data['region'] = (10*(np.ceil((test_data['X'] - min_x)/x_interval)) + (np.ceil((test_data['Y'] - min_y)/y_interval))).astype(int)

print "New max and min values"
print "---------------------------"
print (min_x, max_x, min_y, max_y)

Original max and min values
---------------------------
(-122.51364206429, -120.5, 37.707879022413501, 90.0)
New max and min values
---------------------------
(-122.51364206429, -122.3, 37.707879022413501, 37.799999999999997)


### PARSE THE DATE

In [19]:
train_data.loc[:,'Hour'] = (train_data.loc[:,'Dates']).dt.hour
train_data.loc[:,'Month'] = (train_data.loc[:,'Dates']).dt.month
train_data.loc[:,'Year'] = (train_data.loc[:,'Dates']).dt.year
train_data.loc[:,'Day'] = (train_data.loc[:,'Dates']).dt.day


test_data['Hour'] = (test_data['Dates']).dt.hour
test_data['Month'] = (test_data['Dates']).dt.month
test_data['Year'] = (test_data['Dates']).dt.year
test_data['Day'] = (test_data['Dates']).dt.day



### REMOVE UNUSED COLUMNS

In [ ]:
del train_data['Dates']

In [25]:
del train_data['Descript']
del train_data['X']
del train_data['Y']
del train_data['Address']
del train_data['Resolution']

del test_data['Dates']
del test_data['X']
del test_data['Y']
del test_data['Address']
del test_data['Id']

### MAKE DUMMY VARIABLES

In [26]:
le = preprocessing.LabelEncoder()
days = train_data.loc[:,'DayOfWeek'].unique()
le.fit(days)
train_data.loc[:,'DayOfWeek'] = le.transform(train_data.loc[:,'DayOfWeek']) 


days = test_data.loc[:,'DayOfWeek'].unique()
le.fit(days)
test_data.loc[:,'DayOfWeek'] = le.transform(test_data.loc[:,'DayOfWeek'])


district = train_data.loc[:,'PdDistrict'].unique()
le.fit(district)
train_data.loc[:,'PdDistrict'] = le.transform(train_data.loc[:,'PdDistrict']) 


district = test_data.loc[:,'PdDistrict'].unique()
le.fit(district)
test_data.loc[:,'PdDistrict'] = le.transform(test_data.loc[:,'PdDistrict']) 


district = train_data.loc[:,'region'].unique()
le.fit(district)
train_data.loc[:,'region'] = le.transform(train_data.loc[:,'region']) 


district = test_data.loc[:,'region'].unique()
le.fit(district)
test_data.loc[:,'region'] = le.transform(test_data.loc[:,'region']) 

In [ ]:
'''
# NOT USING THIS - Scale instead.
enc = OneHotEncoder()
train_data_onehot = enc.fit_transform(train_data) 
test_data_onehot = enc.fit_transform(test_data)

# print train_data_onehot.toarray()
print train_data_onehot.shape

#print test_data_onehot.toarray()
print test_data_onehot.shape

#print enc.n_values_
#print enc.feature_indices_
'''

### Scale the data

In [27]:
collist=train_data.columns.tolist()
scaler = preprocessing.StandardScaler()
scaler.fit(train_data)
train_data[collist]=scaler.transform(train_data)

collisttest=test_data.columns.tolist()
scalertest = preprocessing.StandardScaler()
scalertest.fit(test_data)
test_data[collisttest]=scaler.transform(test_data)

# Make train and dev sets

In [41]:
sss = StratifiedShuffleSplit(labels, train_size=0.5)

for train_index, dev_index in sss:
    X_train, X_dev = train_data.iloc[train_index], train_data.iloc[dev_index]
    y_train, y_dev = labels[train_index],labels[dev_index]
    
X_train.index=range(len(X_train))
X_dev.index=range(len(X_dev))

y_train.index=range(len(y_train))
y_dev.index=range(len(y_dev))

train_data.index=range(len(train_data))
labels.index=range(len(labels))


# Classifiers

In [ ]:
classifier = RandomForestClassifier()
model = classifier.fit(X_train, y_train)
print model.score(X_dev, y_dev)

In [ ]:
classifier = DecisionTreeClassifier()
model = classifier.fit(X_train, y_train)
print model.score(X_dev, y_dev)

In [32]:
classifier = LogisticRegression()
model = classifier.fit(X_train, y_train)
print model.score(X_dev, y_dev)

0.202750166276


In [ ]:
classifier = BernoulliNB()
model = classifier.fit(X_train, y_train)
print model.score(X_dev, y_dev)

In [ ]:
classifier = MultinomialNB()
model = classifier.fit(X_train, y_train)
print model.score(X_dev, y_dev)

# Predict and write file

In [ ]:
predictions = model.predict_proba(test_data)
withId  = np.column_stack((map(str,xrange(test_data.shape[0])),predictions))
towrite = np.row_stack((["Id"] + sorted(y_train.unique()),withId))

print(towrite)
write_submission('submission_MultiNB.csv', towrite)

# NEURAL NETWORK

In [33]:
from keras.models import Sequential
from keras.optimizers import SGD
from keras.layers.advanced_activations import PReLU
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils


Using Theano backend.


In [42]:
y_train = y_train.astype('category')
y_train = y_train.cat.rename_categories(range(len(y_train.unique())))

print y_train.shape
print X_train.shape

y_dev = y_dev.astype('category')
y_dev = y_dev.cat.rename_categories(range(len(y_dev.unique())))


(439024,)
(439024, 7)


In [ ]:
output_dim = 39
input_dim = X_train.shape[1]
Nlayers = 1
Nepoch = 20
dp = 0.5



model = Sequential()
model.add(Dense(input_dim=input_dim, output_dim=output_dim, init='glorot_uniform'))
model.add(PReLU(input_shape=(input_dim,)))
model.add(Dropout(dp))

for i in range(Nlayers):
    model.add(Dense(input_dim=input_dim, output_dim=output_dim,init='glorot_uniform'))
    model.add(PReLU(input_shape=(input_dim,)))
    model.add(BatchNormalization())
    model.add(Dropout(dp))

model.add(Dense(input_dim=input_dim, output_dim=output_dim,init='glorot_uniform'))
model.add(Activation('softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    

#model.fit(train_data_onehot.toarray(), y_train, nb_epoch=Nepoch, batch_size=32)
model.fit(X_train.as_matrix() , y_train, nb_epoch=Nepoch, batch_size=32)

Epoch 1/20
265600/439024 [=================>............] - ETA: 10s - loss: 2.7416 - acc: 7.3419e-04